In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
from torch.optim import lr_scheduler
import numpy as np
import time

batch_size = 32

In [28]:
data_dir = "C:/Users/trewe/Desktop/Uni tingz/4 Semester/4 Semester CV/Projekt/data"
train_dir = data_dir + "/train/"

test_dir = data_dir + "/test/"

transform = transforms.Compose([transforms.RandomResizedCrop(48),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_set = datasets.ImageFolder(train_dir, transform = transform)
test_set= datasets.ImageFolder(test_dir,transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=len(test_set), shuffle=False)

In [29]:
classes = train_set.classes
num_classes = len(classes)
print(classes, num_classes)

['happy', 'neutral'] 2


In [30]:
#Hyperparameter
n_epochs = 10
log_interval = 50

#Model
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [31]:
def train(network, train_loader, epoch):
  network.train()
  
  # Loop over the batches
  for batch_idx, (data, target) in enumerate(train_loader):   
    data = data.to(device)
    target = target.to(device)
    optimizer.zero_grad()
    output = network(data) 
    loss = loss_function(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      
      pred = output.data.max(1, keepdim=True)[1]
      correct = pred.eq(target.data.view_as(pred)).sum()
      
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.0f}%'.format(
      epoch, batch_idx * len(data), len(train_loader.dataset),
      100. * batch_idx / len(train_loader), loss.item(), 100. * correct / len(pred)))
      
      train_losses.append(float(loss))

In [32]:
def calc_loss(network, data_loader):

    network.eval()
    correct = 0
    loss = 0
  
    for batch_idx, (data, target) in enumerate(data_loader):     
    # --- Steps of the training of the net ---
        data = data.to(device)
        target = target.to(device)
    
        output = network(data)
        loss = loss_function(output, target)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
    
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    loss, correct, len(data_loader.dataset),
    100. * correct / len(data_loader.dataset)))
    return float(loss)

In [33]:
train_losses = []
test_losses = []

start_time = time.time()

test_loss = calc_loss(model, test_loader)
test_losses.append(test_loss)
print("Test CrossEntropyLoss: ", float(test_loss))
for i in range(n_epochs):
    #torch.cuda.empty_cache()
    print("epoche", i)
    
    train(model,train_loader,i)
    test_loss = calc_loss(model, test_loader)
    test_losses.append(test_loss)
    
    
    print("Test CrossEntropyLoss: ", float(test_loss))
    print("-----------------")
    
    
print("******************")
print("nach,", n_epochs ," Epochen: ")
print("Train CrossEntropyLoss: ", train_losses[-1])
print("Test CrossEntropyLoss: ", test_losses[-1])
print("--- %s seconds ---" % (time.time() - start_time))


Test set: Avg. loss: 0.9623, Accuracy: 1243/3007 (41%)

Test CrossEntropyLoss:  0.9623400568962097
epoche 0
Train Epoch: 0 [0/7732 (0%)]	Loss: 0.916679	Accuracy: 41%
Train Epoch: 0 [1600/7732 (21%)]	Loss: 0.685197	Accuracy: 56%
Train Epoch: 0 [3200/7732 (41%)]	Loss: 0.559982	Accuracy: 62%
Train Epoch: 0 [4800/7732 (62%)]	Loss: 0.450275	Accuracy: 78%
Train Epoch: 0 [6400/7732 (83%)]	Loss: 0.642666	Accuracy: 62%

Test set: Avg. loss: 0.4753, Accuracy: 2226/3007 (74%)

Test CrossEntropyLoss:  0.47534018754959106
-----------------
epoche 1
Train Epoch: 1 [0/7732 (0%)]	Loss: 0.474394	Accuracy: 72%
Train Epoch: 1 [1600/7732 (21%)]	Loss: 0.489298	Accuracy: 78%
Train Epoch: 1 [3200/7732 (41%)]	Loss: 0.361956	Accuracy: 88%
Train Epoch: 1 [4800/7732 (62%)]	Loss: 0.248546	Accuracy: 94%
Train Epoch: 1 [6400/7732 (83%)]	Loss: 0.456815	Accuracy: 78%

Test set: Avg. loss: 0.6219, Accuracy: 1946/3007 (65%)

Test CrossEntropyLoss:  0.6218953728675842
-----------------
epoche 2
Train Epoch: 2 [0/7732 (

In [34]:
torch.save(model.state_dict(), "./model.pt")
torch.save(optimizer.state_dict(), "./model_opt.pt")
